### Drop Columns

In [ ]:
#The purpose is to predict the delay, so Dataset "airlines" and "airports" will be process later as labels.

#Clean up the main dataset "flights"
columns = flights.columns
print("Features of current dataset",columns)

#Clean up the non-related data
#Drop the reason of delaying / Cancellation_reasons / cancelled / diverted etc.
#Reason of dropping, please see README
flights=flights.drop(['WEATHER_DELAY','LATE_AIRCRAFT_DELAY','AIRLINE_DELAY','SECURITY_DELAY','AIR_SYSTEM_DELAY','CANCELLATION_REASON',
 "CANCELLED", "DIVERTED","TAXI_IN", "TAXI_OUT"],axis=1)

columns = flights.columns
print("Features after cleaning unrelated dataset",columns)

In [ ]:
#Clean up the overlap numbers, reasons please see the README
flights=flights.drop(['SCHEDULED_ARRIVAL','ARRIVAL_TIME','DEPARTURE_TIME','SCHEDULED_DEPARTURE','TAIL_NUMBER','FLIGHT_NUMBER', 'WHEELS_OFF', 'WHEELS_ON', "SCHEDULED_TIME", "ELAPSED_TIME"],axis=1)

# Since all data is collected from 2015, year is meaningless to help prediction
print(flights["YEAR"].unique())
flights=flights.drop(['YEAR'],axis=1)

In [ ]:
columns = flights.columns
print("Features after cleaning unrelated dataset",columns)
flights.shape

In [ ]:
print(flights.dtypes)
flights

### Encoding Categorial Feature

In [ ]:
num_flights = flights.loc[:,['DEPARTURE_DELAY',"AIR_TIME","DISTANCE"]]
one_hot_original = flights.loc[:,['AIRLINE',"DAY_OF_WEEK","MONTH"]]
ordinal_orginal = flights.loc[:,['DAY',"ORIGIN_AIRPORT","DESTINATION_AIRPORT"]]

display(one_hot_original)
encoder = preprocessing.OneHotEncoder()
encoder.fit(one_hot_original)
one_hot = encoder.transform(one_hot_original)
print(one_hot.shape)


display(one_hot_original)
one_hot_encode=one_hot_original.astype("str")
encoder = preprocessing.OrdinalEncoder()
encoder.fit(one_hot_encode)
ordinal_cat = encoder.transform(one_hot_encode)
print(ordinal_cat, ordinal_cat.shape)

display(ordinal_orginal)
ordinal_orginal["DAY"] = ordinal_orginal["DAY"].astype("str")
ordinal_orginal.dtypes
ordinal_orginal=ordinal_orginal.astype("str")
#ordinal_orginal.dtypes
encoder = preprocessing.OrdinalEncoder()
encoder.fit(ordinal_orginal)
ordinal = encoder.transform(ordinal_orginal)
print(ordinal, ordinal.shape)

df_ordinal = pd.DataFrame(ordinal, columns = ['DAY','ORIGIN_AIRPORT','DESTINATION_AIRPORT'])
df_cat = pd.DataFrame(ordinal_cat, columns = ['AIRLINE','DAY_OF_WEEK','MONTH'])

df_flights = num_flights.join(df_ordinal)
df_flights = df_flights.join(df_cat)
df_flights

### Figure Display after Encoding

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = df_flights.corr()
sns.heatmap(corr, annot = True, vmin=-1, vmax=1, center= 0,cmap= 'RdBu')

In [ ]:
sns.pairplot(data=df_flights.sample(n=8000),
             vars=df_flights.columns,
             diag_kind='kde',
             plot_kws=dict(
                 size=.5,
                 alpha=.5,
             ))

### Scaler

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_flights)
df_flight_noscale =df_flights
df_flights = scaler.transform (df_flights)

# Make df_flights into Dataframe
df_flights = pd.DataFrame(df_flights)
df_flights = df_flights.rename(columns={0:'DEPARTURE_DELAY',1:'AIR_TIME',2:'DISTANCE',3:'DAY',4:'ORINGIN_AIRPORT',5:'DESTINATION_AIRPORT',6:'AIRLINES',7:'DAY_OF_WEEK',8:'MONTH' })

target = flights["ARRIVAL_DELAY"]
df_flights = df_flights.join(target)
df_flights

###  Clean up NAN Values

In [ ]:
# "DEPARTURE_DELAY", "AIR_TIME"
# When there is no departure_delay and arrival dealy, value would be None
# Under this condition, set these NAN values as 0

# check NAN values
display(df_flights)
display(df_flights.isna().any())

df_flights['DEPARTURE_DELAY'] = df_flights['DEPARTURE_DELAY'].fillna(0)
df_flights['ARRIVAL_DELAY'] = df_flights['ARRIVAL_DELAY'].fillna(0)

print("After filling DEPARTURE_DELAY, and ARRIVAL_DELAY:")
display(df_flights.isna().any())

In [ ]:
#Drop rows where the air_time is NAN. (It is noise on Air_time record)
# 0.18% of overall samples is dropped
df_flights = df_flights.dropna()
display(df_flights)
print("after deleting all rows that contain NaN values,")
display(df_flights.isna().any())